# Visualize calcium for a target experiment

In [ ]:
# imports

import logging
logging.getLogger("param.Dimension").setLevel(logging.CRITICAL)
logging.getLogger("param.ParameterizedMetaclass").setLevel(logging.CRITICAL)
logging.getLogger("param.HistogramPlot").setLevel(logging.CRITICAL)
logging.getLogger("param.AdjointLayout").setLevel(logging.CRITICAL)
logging.getLogger("param.OverlayPlot").setLevel(logging.CRITICAL)
logging.getLogger("param.HoloMap").setLevel(logging.CRITICAL)
logging.getLogger("param.CurvePlot").setLevel(logging.CRITICAL)
logging.getLogger("param.Layout").setLevel(logging.CRITICAL)
logging.getLogger("param.LayerPlot").setLevel(logging.CRITICAL)
logging.getLogger("param.RasterPlot").setLevel(logging.CRITICAL)
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))
import panel as pn
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')
from bokeh.resources import INLINE

import importlib
import processing_parameters
import paths
import functions_bondjango as bd
import functions_misc as fm
import functions_plotting as fp
import pandas as pd
import numpy as np
import sklearn.mixture as mix
import sklearn.decomposition as decomp
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn import svm, datasets
from sklearn import preprocessing
import sklearn.linear_model as lin
import h5py

import random
import functions_data_handling as fd

In [ ]:
# Load the desired files
importlib.reload(processing_parameters)
# get the data paths
try: 
    data_path = snakemake.input[0]
except NameError:
    # define the search string
    search_string = processing_parameters.search_string
    # query the database for data to plot
    data_all = bd.query_database('analyzed_data', search_string)
    data_path = [el['analysis_path'] for el in data_all if 'preproc' in el['slug']]
    print(data_path)
    # load the calcium data
    beh_data = []
    data = []
    # for all the files
    for files in data_path:
        # load the data
        with pd.HDFStore(files) as h:
            beh_data.append(h['full_traces'])
            if '/matched_calcium' in h.keys():
                data.append(h['matched_calcium'])
                print(h['matched_calcium'].shape)
    # concatenate the data
    data = pd.concat(data)

In [ ]:
# Plot the responses

# get just the calcium traces
calcium_data = np.array([data.loc[:, el].to_numpy() for el in data.columns if 'cell' in el])
# print(calcium['cell_1'])
# print(calcium_data)
# plot

hv.Image(calcium_data, kdims=['Time', 'ROI'], vdims='dF/F').opts(width=800, height=400, cmap='viridis')

In [ ]:
# plot an ethogram

In [ ]:
# plot a sorted correlation matrix

# define the variables to use for correlation
variables = ['mouse_speed', 'mouse_acceleration', 'cricket_0_mouse_distance', 'cricket_0_delta_heading',
            'cricket_0_speed', 'cricket_0_acceleration', 'cricket_0_visual_angle',
            'cricket_0_quadrant']
# get the number of behavioral variables
behavioral_number = len(variables)
# get the data
behavioral_data = data.loc[:, variables].to_numpy()

# get the number of ROIs
roi_number = calcium_data.shape[0]

# calculate the correlation matrix
corr_matrix = np.corrcoef(calcium_data, behavioral_data.T)

# sort the matrix based on the correlation values
cell_matrix = corr_matrix[:roi_number, -behavioral_number:]
idx = np.argsort(cell_matrix[:, 0])
cell_matrix = cell_matrix[idx, :].T

y_labels = [(-(idx+0.5)/(len(variables))+0.5, el) for idx, el in enumerate(variables)]
print(y_labels)
# plot the matrix
hv.Image(cell_matrix, kdims=['ROIs', 'Variables']).opts(width=800, height=600, tools=['hover'], cmap='viridis', yticks=y_labels,
                          colorbar=True)

In [ ]:
# Calculate the per cell rev correlation

# define the interval for calculation in frames
target_interval = 50
# define the target behavioral variable
target_behavior = 'cricket_0_mouse_distance'
# get the number of time points
time_number = calcium_data.shape[1]

# get the behavioral data
current_behavior = np.expand_dims(data.loc[:, target_behavior].to_numpy(), axis=1)
# pad the behavior data
padded_behavior = np.vstack((np.zeros((target_interval, 1)), current_behavior)).T
# padded_behavior = np.vstack((np.zeros((target_interval, 1)), current_behavior, 
#                              np.zeros((target_interval, 1)))).T
# allocate memory to save the ols
ols_list = []
prediction_list = []
# for all the cells
for cells in np.arange(roi_number):
    # allocate memory for the padded matrix
    time_matrix = np.zeros((time_number, target_interval))
#     time_matrix = np.zeros((time_number, 2*target_interval))
    # get the calcium data
    current_calcium = calcium_data[cells:cells+1, :].T

    # fill the padded matrix
    for frame, el in enumerate(current_calcium):
        time_matrix[frame, :] = padded_behavior[:, frame:frame+target_interval]
#         time_matrix[frame, :] = padded_behavior[:, frame:frame+2*target_interval]
#     # calculate the covariance matrix
#     cov_matrix = time_matrix.T@time_matrix
#     # calculate the STA
#     sta = time_matrix.T@current_calcium
#     # calculate the ols estimate
#     ols = np.linalg.inv(cov_matrix) @ sta   
    
    ols = lin.PoissonRegressor(alpha=.1, max_iter=5000).fit(time_matrix, current_calcium.ravel())
    
    # save the ols
    ols_list.append(ols.coef_)
    prediction_list.append(ols.predict(time_matrix))

In [ ]:
# Plot the OLS

# allocate memory for the cells
cell_dict = {}
prediction_dict = {}
print(ols_list[0].shape)

# generate the x axis vector
x = range(target_interval)
x_pred = range(current_behavior.shape[0])
# x = range(2*target_interval)
# for all the cells
for cells in np.arange(roi_number):
    
    # generate the plot and save
    cell_plot = hv.Curve((x, ols_list[cells].flatten()), kdims=['Time'], vdims=target_behavior)
    cell_plot.opts(shared_axes=False, width=600)
    
    current_calcium = calcium_data[cells:cells+1, :].ravel()
    prediction_plot = hv.Curve((x_pred, current_calcium)) * \
        hv.Curve((x_pred, prediction_list[cells]))
    prediction_plot.opts(shared_axes=False)
    # save in a dict
    cell_dict[cells] = cell_plot
    prediction_dict[cells] = prediction_plot

cell_holomap = hv.HoloMap(prediction_dict)
cell_holomap

In [ ]:
# old ethogram code

# allocate a dict for the holomap
holo_dict = {}
# select the behavioral variables to show
target_behavior = ['cricket_0_mouse_distance','mouse_speed','cricket_0_speed','cricket_0_delta_head']
# define the plot offset
offset = 1
# define the target mouse, date and trial
mouse = 'MM_191108_a'
day = '2019_12_11'
trial = 0


# # get the table
# if isinstance(data, dict):
#     sub_data = data[mouse][day]
# else:
#     sub_data = data
sub_data = data

# get the available columns
labels = list(sub_data.columns)
# print(labels)
cells = [el for el in labels if 'cell' in el]
# calculate the average neuronal response
# sub_data = sub_data.loc[sub_data['trial_id']==0, target_behavior + cells]
sub_data = sub_data.loc[:, target_behavior + cells]
sub_data = sub_data.loc[5:, :]

# separate the cells and non-cells
behavior_data = sub_data[target_behavior]
sub_data = sub_data[cells]
# get the column labels
labels = sub_data.columns
# get the number of rows
x_axis = sub_data.shape[0]
# get the array
sub_array = sub_data.to_numpy().T
sub_array = fm.normalize_matrix(sub_array, axis=1)

# plot the behavioral variables
traces = []
# get the x axis values
x = np.array(np.arange(x_axis))
# get the labels of the behavioral variables
behavior_labels = behavior_data.columns

# replace the delta head name for heading
behavior_labels = ['heading' if el=='cricket_0_delta_head' else el for el in behavior_labels]
behavior_labels = [el.replace('cricket_0_', 'cricket_') if 'cricket_0_' in el else el for el in behavior_labels]

# define the ticks
y_tick = [(idx-0.5, el.replace('_',' ')) for idx, el in enumerate(behavior_labels)]
# for all the behavioral variables
for idx, variable in enumerate(target_behavior):
    
        # if it's the heading
    if variable == 'cricket_0_delta_head':
        # draw a line through the middle of the heading parameter
        trace2 = hv.Curve((x, 0.5+offset*(idx-1)), 'Time',variable)
        trace2.opts(width=800, height=100, xticks=0, xlabel='', yticks=0, ylabel='', 
               toolbar=None, color='red', line_dash='dotted', line_width=2)
        
        traces.append(trace2)
    
    y = behavior_data[variable].to_numpy()
    if variable != 'cricket_0_delta_head':
        y = np.log(y)
        y[np.isnan(y)] = 0
        y[np.isinf(y)] = 0
            
    y = fm.normalize_matrix(y, axis=0)+offset*(idx-1)
    
    trace = hv.Curve((x, y), 'Time',variable)
#     trace = hv.Curve((x, behavior_data[variable].to_numpy()+offset*(idx-1)), 'Time',variable)
    trace.opts(width=800, height=100, xticks=0, xlabel='', yticks=y_tick, ylabel='', 
               toolbar=None, color='black', line_width=2)
#     if variable == 'mouse_speed':
#         print(fm.normalize_matrix(np.log(behavior_data[variable].to_numpy()), axis=0))

    # append
    traces.append(trace)
    

    
# plot the cells
raw_image = hv.Image(sub_array, ['Time (s)','Cells'], bounds=[0,-0.5,x_axis,0.5], fontstyle='regular')

raw_image.opts(width=800, height=600, colorbar=True, colorbar_opts={'major_label_text_align': 'left'},
               cmap='Viridis', yticks=[(-0.5, 1), (0.5, sub_array.shape[0])], toolbar=None,
               fontsize=15, hooks=[fp.margin])

# create the overlay with the behavioral traces
overlay = hv.Overlay(traces)
overlay.opts(text_font='Arial',fontsize=15)

# create the final layout
layout = hv.Layout(raw_image+overlay).cols(1)
layout.opts(toolbar=None, fontsize=10)

# assemble the save path
# save_path = os.path.join(paths.figures_path,'_'.join((save_name,mouse,day,str(trial),'ethogram.png')))
# hv.save(layout,save_path)

layout